# MERCS 101 - Lecture 01: Classification

This is the first part of the tutorial, focusing on MERCS as a simple classifier.

## Preliminaries

### External Imports

In [1]:
import numpy as np
import os
import sys
from sklearn.metrics import f1_score, accuracy_score, classification_report
import pandas as pd

### MERCS imports

In [2]:
sys.path.insert(0, '..') # We add the parent dir to the path
from src.mercs.core import MERCS
from src.mercs.utils import *

import src.datasets as datasets

## Induction

### Importing Data

First, we import the nursery dataset.

In [3]:
train, test = datasets.load_nursery()

load_example_dataset is loading fname: ../resc/data/nursery_train.csv

load_example_dataset is loading fname: ../resc/data/nursery_test.csv



This is a fully nominal dataset

In [4]:
train.head()

,Var0,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8
0,2,3,0,0,0,0,0,2,2
1,2,3,0,0,0,0,0,1,1
2,2,3,0,0,0,0,0,0,0
3,2,3,0,0,0,0,2,2,2
4,2,3,0,0,0,0,2,1,1


### Training

In [5]:
model = MERCS()

In [6]:
ind_parameters = {'ind_type':           'RF',
                  'ind_n_estimators':   30}

sel_parameters = {'sel_type':           'Base',
                  'sel_its':            4,
                  'sel_param':          2}

In [7]:
model.fit(train, **ind_parameters, **sel_parameters)

## Inference

### Prediction

In [8]:
code = [0,0,0,0,0,0,0,1,1]
len(code)

9

In [9]:
pred_parameters = {'pred_type':    'MI',
                   'pred_param':   1.0,
                   'pred_its':     8}

In [10]:
y_pred = model.predict(test,
                       **pred_parameters,
                       qry_code=code)

SETTINGS.PY: I AM READING A SINGLE QUERY CODE, I.E: [0, 0, 0, 0, 0, 0, 0, 1, 1]
Predicting q_code: [0, 0, 0, 0, 0, 0, 0, 1, 1]


In [11]:
y_pred

array([[2., 4.],
       [0., 0.],
       [2., 4.],
       ...,
       [2., 1.],
       [1., 3.],
       [0., 0.]])

### Evaluation 

In [12]:
y_true = test[test.columns.values[np.array(code)==1]].values

In [13]:
obs = f1_score(y_true, y_pred, average='macro')
obs

ValueError: multiclass-multioutput is not supported

In [ ]:
assert isinstance(obs, (int, float))
assert 0 <= obs <= 1